In [23]:
%%HTML
<style>
    body {
        --vscode-font-family: "Noto Serif"
    }
</style>

# Prefix Sum Primer: What, When, Patterns, and Problems



## What is Prefix Sum?


- For an array $a$ of length $n$, define a prefix sum array $P$ of length $n+1$:
  - $P[0] = 0$
  - $P[i] = \sum_{k=0}^{i-1} a[k]$ for $i \in [1, n]$
- Any subarray sum from $i$ to $j$ (0-based, inclusive) is:
  - $\text{sum}(i, j) = P[j+1] - P[i]$
- Build once in $O(n)$, then range sum queries are $O(1)$.


Minimal Python template:
```python
def prefix_sum(a):
    P = [0]*(len(a)+1)
    for i, x in enumerate(a, 1):
        P[i] = P[i-1] + x
    return P


def range_sum(P, i, j):  # inclusive
    return P[j+1] - P[i]
```



## When to use it


- Many range-sum queries after one precompute.
- Count subarrays with a given property by mapping prefix values.
- Replace nested loops and reduce complexity from $O(n^2)$ to $O(n)$.
- 2D grids for rectangle sums via inclusion–exclusion.
- Efficient range updates via a difference array (inverse of prefixing).




## Pitfalls and tips


- Off-by-one: Using length $n+1$ with $P[0]=0$ simplifies queries.
- Negatives: Sliding window breaks; prefix+hashmap works.
- Modulo with negatives: Normalize where languages have negative remainders.
- 2D borders: $(m+1)\times(n+1)$ prefix avoids conditional edges.

## Core patterns implementation

- 1D template and O(1) range queries
- Counting subarrays with target sum (hashmap)
- Subarrays with sum divisible by K (mod trick)
- 2D prefix sums for fast rectangle queries
- Difference array for efficient range updates


### Range query

In [24]:
# Range Sum with Prefix Sums (descriptive variable names)
from typing import List, Tuple

def build_prefix(nums: List[int]) -> List[int]:
    """Return prefix sums array prefix_sums where prefix_sums[i] = sum(nums[:i]).
    Length is len(nums)+1 and prefix_sums[0] = 0 for convenient O(1) range queries.
    """
    prefix_sums: List[int] = [0] * (len(nums) + 1)
    for idx, value in enumerate(nums, start=1):
        prefix_sums[idx] = prefix_sums[idx - 1] + value
    return prefix_sums

def range_sum(prefix_sums: List[int], left: int, right: int) -> int:
    """Return sum of nums[left..right] inclusive using precomputed prefix_sums."""
    return prefix_sums[right + 1] - prefix_sums[left]

# quick smoke test
sample_array = [2, -1, 3, 4]
prefix_sums = build_prefix(sample_array)
assert range_sum(prefix_sums, 0, 2) == 4  # 2-1+3
assert range_sum(prefix_sums, 1, 3) == 6  # -1+3+4
prefix_sums, range_sum(prefix_sums, 1, 3)

([0, 2, 1, 4, 8], 6)

### 1) Subarray sum equals K (hashmap on prefix)

Use the identity: if $P[j] - P[i] = K$, then $P[i] = P[j] - K$.


In [25]:
# Subarray Sum Equals K (descriptive variable names)
from collections import defaultdict
from typing import List

def subarray_sum_equals_k(nums: List[int], target_sum: int) -> int:
    # freq[prefix_value] = count of occurrences of that prefix sum so far
    prefix_frequency = defaultdict(int)
    prefix_frequency[0] = 1  # empty prefix contributes 0 once
    running_prefix = 0
    total_subarrays = 0
    for value in nums:
        running_prefix += value
        # We want running_prefix - previous_prefix = target_sum -> previous_prefix = running_prefix - target_sum
        total_subarrays += prefix_frequency[running_prefix - target_sum]
        prefix_frequency[running_prefix] += 1
    return total_subarrays

# smoke tests
assert subarray_sum_equals_k([1,1,1], 2) == 2
assert subarray_sum_equals_k([1,-1,0], 0) == 3
subarray_sum_equals_k([3,4,7,2,-3,1,4,2], 7)

4

Notes:
- Works with negatives (where sliding window fails).
- Time $O(n)$, Space $O(n)$.

### 2) Subarray sums divisible by K (mod trick)

If $P[i] \bmod K = P[j] \bmod K$, then $\sum_{t=i}^{j-1} a[t]$ is divisible by $K$.


In [26]:
# Subarrays Divisible by K (descriptive variable names)
from collections import defaultdict
from typing import List

def subarrays_divisible_by_k(nums: List[int], k: int) -> int:
    # freq[mod_value] = count of prefix sums with this modulo k
    modulo_frequency = defaultdict(int)
    modulo_frequency[0] = 1  # empty prefix has modulo 0
    running_mod = 0
    total_subarrays = 0
    for value in nums:
        running_mod = (running_mod + value) % k
        total_subarrays += modulo_frequency[running_mod]
        modulo_frequency[running_mod] += 1
    return total_subarrays

# smoke tests
assert subarrays_divisible_by_k([4,5,0,-2,-3,1], 5) == 7
subarrays_divisible_by_k([1,2,3,4,5], 3)

7

### 3) 2D prefix sums (matrix region sums)

Define an $(m+1)\times(n+1)$ matrix $S$ with zeros on row 0 and column 0:
- $S[i][j] = \sum_{r=0}^{i-1} \sum_{c=0}^{j-1} M[r][c]$
- For rectangle rows $[r_1..r_2]$, cols $[c_1..c_2]$ (inclusive):
  - $\text{sum} = S[r_2+1][c_2+1] - S[r_1][c_2+1] - S[r_2+1][c_1] + S[r_1][c_1]$


In [27]:
# 2D Prefix Sum (descriptive variable names)
from typing import List

def build_2d_prefix(matrix: List[List[int]]):
    row_count = len(matrix)
    col_count = len(matrix[0]) if matrix else 0
    prefix2d = [[0] * (col_count + 1) for _ in range(row_count + 1)]
    for r in range(1, row_count + 1):
        row_running_sum = 0
        for c in range(1, col_count + 1):
            row_running_sum += matrix[r - 1][c - 1]
            prefix2d[r][c] = prefix2d[r - 1][c] + row_running_sum
    return prefix2d

def sum_region(prefix2d, top_row: int, left_col: int, bottom_row: int, right_col: int):
    return (
        prefix2d[bottom_row + 1][right_col + 1]
        - prefix2d[top_row][right_col + 1]
        - prefix2d[bottom_row + 1][left_col]
        + prefix2d[top_row][left_col]
    )

# smoke test
matrix_example = [
    [3, 0, 1, 4, 2],
    [5, 6, 3, 2, 1],
    [1, 2, 0, 1, 5],
    [4, 1, 0, 1, 7],
    [1, 0, 3, 0, 5],
]
prefix2d = build_2d_prefix(matrix_example)
assert sum_region(prefix2d, 2, 1, 4, 3) == 8
sum_region(prefix2d, 1, 1, 2, 2)

11

### 4) Difference array (range updates)
To add $v$ to every $a[l..r]$, maintain a difference array $D$:
- $D[l] \mathrel{+}= v$
- $D[r+1] \mathrel{-}= v$ if $r+1 < n$
- The final array is the prefix sum of $D$.


In [28]:
# Apply Range Updates (Difference Array with descriptive names)
from typing import List, Tuple

def apply_range_updates(length: int, updates: List[Tuple[int, int, int]]) -> List[int]:
    """Apply inclusive range updates (start, end, delta) using a difference array.
    Returns the final array after all updates.
    """
    difference = [0] * (length + 1)
    for start_idx, end_idx, delta in updates:
        difference[start_idx] += delta
        if end_idx + 1 < length:
            difference[end_idx + 1] -= delta
    result = [0] * length
    running_total = 0
    for i in range(length):
        running_total += difference[i]
        result[i] = running_total
    return result

# smoke tests
assert apply_range_updates(5, [(1,3,2)]) == [0,2,2,2,0]
assert apply_range_updates(5, [(0,4,1), (2,4,3)]) == [1,1,4,4,4]
apply_range_updates(10, [(0,0,5), (5,9,1), (3,7,2)])

[5, 0, 0, 2, 2, 3, 3, 3, 1, 1]